In [ ]:
import numpy as np
import pandas as pd

import os

import sklearn
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras

import multiprocessing

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.applications import EfficientNetB0

import cv2

from skimage.transform import resize
import numpy as np
import math

In [ ]:
GLOBAL_SEED = 42

np.random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)

In [ ]:
train = pd.read_csv("../input/hotel-id-2021-fgvc8/train.csv")

In [ ]:
kaggle_path = "../input/hotel-id-2021-fgvc8/train_images/"
train['full_filepath'] = kaggle_path + train.chain.astype(str) +"/"+ train.image.astype(str)

In [ ]:
train.head()

In [ ]:
train = train[train.chain.isin([0,1,2])]
train.shape

In [ ]:
n_subsample = 5000
train = train.sample(n_subsample)

In [ ]:
X_train, X_val, = train_test_split(train, test_size = 0.30,
    stratify = train['chain'], random_state = GLOBAL_SEED, shuffle = True
)

In [ ]:
print(X_train.shape)
print(X_val.shape)

In [ ]:
n_classes = X_train.chain.nunique()

BATCH_SIZE = 64
STEPS_PER_EPOCH = len(X_train) // BATCH_SIZE
EPOCHS = 50

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)

In [ ]:
n_classes

In [ ]:
class HotelBatchSequence(tf.keras.utils.Sequence):
    
    def __init__(self, x_set, y_set, batch_size,
                 img_size = (224, 224),
                 augment = False):
        """
        `x_set` is list of paths to the images
        `y_set` are the associated classes.

        """
        
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.img_size = img_size
    
    def __len__(self):
        """Denotes the number of batches per epoch"""
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        """Generate one batch of data"""
        
        first_id = idx * self.batch_size
        last_id =  (idx + 1) * (self.batch_size)
        
        batch_x = self.x[first_id:last_id]
        batch_y = self.y[first_id:last_id]
        
        #Xs = np.array([resize(imread(file_name), self.img_size)
        #      for file_name in batch_x])
        # 
        #ys = np.array(batch_y)
        
        output = np.array([
            resize(cv2.imread(file_name), self.img_size)
                   for file_name in batch_x]), np.array(batch_y)
        
        return output

In [ ]:
TrainGenerator = HotelBatchSequence(X_train.full_filepath, 
                                    tf.keras.utils.to_categorical(X_train.chain),
                                    BATCH_SIZE)

ValidGenerator = HotelBatchSequence(X_val.full_filepath, 
                                   tf.keras.utils.to_categorical(X_val.chain),
                                   BATCH_SIZE)

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

model.compile(loss='categorical_crossentropy',
  optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(
    TrainGenerator, 
    validation_data=ValidGenerator,
    batch_size=50,
    epochs=2)

In [ ]:
model = EfficientNetB0(include_top=True, 
                              weights=None, 
                              input_shape = (IMG_HEIGHT, IMG_WIDTH, 3),
                              classes = n_classes
)

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(TrainGenerator,
          batch_size=128,
          validation_data = ValidGenerator,
          epochs = 2)

In [ ]:
prediction = model.predict([prepare('99e91ad5f2870678.jpg')])

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
history = model.fit(TrainGenerator,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    validation_data = ValidGenerator,
                    epochs = 2)